In [47]:
import pandas as pd
import numpy as np
import datetime

In [48]:
mobility = pd.read_csv('2020_US_Region_Mobility_Report.csv') #read in data
covid = pd.read_excel('covid-19.xlsx', sheet_name = 'Covid-19', header = 1).set_index('State')
covid.head()

,Total Cases,Cases in Last 7 Days,Case Rate per 100000,Case Rate per 100000 in Last 7 Days,Total Deaths,Deaths in Last 7 Days,Death Rate per 100000,Death Rate per 100K in Last 7 Days,# Tests Performed,% Positive,...,Adults 26-34,Adults 35-54,Adults 55-64,65+,Retail & recreation,Grocery & pharmacy,Parks,Transit stations,Workplaces,Residential
State,,,,,,,,,,,,,,,,,,,,,
Alabama,189149,12085,3870,35.3,2914,71,59,0.2,2103005,11-20%,...,0.12,0.25,0.14,0.17,-11,-6,20,-8,-22,5
Alaska,14456,2619,1960,50.7,77,9,10,0.2,795937,0-5%,...,0.13,0.26,0.13,0.12,-9,-7,21,-26,-24,10
Arizona,242480,7574,3381,15.1,5918,59,82,0.1,2356325,6-10%,...,0.12,0.24,0.12,0.18,-18,-11,-11,-21,-33,9
Arkansas,109712,6914,3640,32.8,1894,122,62,0.6,1304131,6-10%,...,0.12,0.25,0.13,0.17,-7,-4,66,-3,-22,5
California,912904,32180,2308,11.6,17541,352,44,0.1,18098631,6-10%,...,0.14,0.26,0.12,0.14,-26,-11,-12,-39,-36,12


In [102]:
mobility['date'] = pd.to_datetime(mobility['date']) #convert str object to datetime
date_after = pd.to_datetime('2020-09-12') #we don't anticipate any earlier mobility rates to be as explanatory of 10/30 outcome
date_before = pd.to_datetime('2020-10-30') #cutoff is 10/30 since all outcome variables are also captured on 10/30/3030
mobility = mobility[mobility['date'] >= date_after]
mobility = mobility[mobility['date'] <= date_before]

mobility = mobility.rename(columns={"retail_and_recreation_percent_change_from_baseline": "retail_rec", 
                                    "grocery_and_pharmacy_percent_change_from_baseline": "groc_pharm",
                                   "parks_percent_change_from_baseline": "parks",
                                   "transit_stations_percent_change_from_baseline": "transit",
                                   "workplaces_percent_change_from_baseline": "workplaces",
                                   "residential_percent_change_from_baseline": "residential"})
mobility.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_rec,groc_pharm,parks,transit,workplaces,residential
211,US,United States,NaN,NaN,NaN,NaN,NaN,2020-09-13,-16.0,-6.0,38.0,-25.0,-15.0,3.0
212,US,United States,NaN,NaN,NaN,NaN,NaN,2020-09-14,-16.0,-7.0,30.0,-33.0,-32.0,8.0
213,US,United States,NaN,NaN,NaN,NaN,NaN,2020-09-15,-14.0,-5.0,36.0,-32.0,-33.0,9.0
214,US,United States,NaN,NaN,NaN,NaN,NaN,2020-09-16,-15.0,-8.0,36.0,-32.0,-33.0,9.0
215,US,United States,NaN,NaN,NaN,NaN,NaN,2020-09-17,-16.0,-7.0,34.0,-32.0,-33.0,9.0


In [103]:
#helper function to rename the columns in pivot table - becomes helpful for the merge later on
def rename_cols(col_list, column_name):
    names = {}
    for i in col_list:
        names[i] = column_name + "_" + str(i)[:10]  
    return names
    
#function makes pivot tables to get the df in correct shape for each category
def make_pivot(mobility, column_name):
    df = mobility[['sub_region_1', 'date', column_name]]
    df['Date'] = pd.to_datetime(df['date']) - pd.to_timedelta(7, unit='d')
    df = df.groupby(['sub_region_1', pd.Grouper(key='Date', freq='W-SAT')])[column_name].mean().reset_index().sort_values('Date')
    df = df.pivot(index='sub_region_1', columns='Date', values = column_name)
    col_dict = rename_cols(df.columns, column_name)
    df.rename(columns = col_dict, inplace = True)
    return df


In [104]:
df_rec = make_pivot(mobility, 'retail_rec')
df_groc_pharm = make_pivot(mobility, 'groc_pharm')
df_parks = make_pivot(mobility, 'parks')
df_transit = make_pivot(mobility, 'transit')
df_workplaces = make_pivot(mobility, 'workplaces')
df_residential = make_pivot(mobility, 'residential')

<ipython-input-103-c07ecc17081b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['date']) - pd.to_timedelta(7, unit='d')


In [105]:
#df_rec.index
mobility_cleaned = df_rec.join(df_groc_pharm).join(df_parks).join(df_transit).join(df_workplaces).join(df_residential)
mobility_cleaned = covid.join(mobility_cleaned)
mobility_cleaned = mobility_cleaned.reset_index()

mobility_cleaned.to_csv("mobility_data.csv")

In [106]:
mobility_cleaned.head()


,State,Total Cases,Cases in Last 7 Days,Case Rate per 100000,Case Rate per 100000 in Last 7 Days,Total Deaths,Deaths in Last 7 Days,Death Rate per 100000,Death Rate per 100K in Last 7 Days,# Tests Performed,...,workplaces_2020-10-10,workplaces_2020-10-17,workplaces_2020-10-24,residential_2020-09-12,residential_2020-09-19,residential_2020-09-26,residential_2020-10-03,residential_2020-10-10,residential_2020-10-17,residential_2020-10-24
0,Alabama,189149,12085,3870,35.3,2914,71,59,0.2,2103005,...,-18.864865,-17.114865,-20.769231,3.904762,4.188525,2.457875,3.155797,3.797048,3.465201,2.774590
1,Alaska,14456,2619,1960,50.7,77,9,10,0.2,795937,...,-16.042254,-16.605634,-16.952381,3.606061,5.305556,5.230769,5.743590,7.230769,7.846154,8.823529
2,Arizona,242480,7574,3381,15.1,5918,59,82,0.1,2356325,...,-26.918182,-23.709091,-24.715789,5.898876,6.552083,5.732673,5.475248,6.217822,5.772277,6.409091
3,Arkansas,109712,6914,3640,32.8,1894,122,62,0.6,1304131,...,-16.368182,-15.986364,-16.508816,2.882353,4.411392,2.405714,2.807910,3.537143,3.744318,4.967949
4,California,912904,32180,2308,11.6,17541,352,44,0.1,18098631,...,-25.605598,-24.559796,-26.592920,8.567308,8.226300,7.805882,7.591176,7.817647,7.732353,7.881356
